In [3]:
import sys
sys.path.append("../../../..")
import os
#os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
%load_ext autoreload
%autoreload 1
import lightning.pytorch as pl
from lightning.pytorch.callbacks import DeviceStatsMonitor
from lightning.pytorch.tuner.tuning import Tuner
import numpy as np

from ntupleReaders.clue_ntuple_reader import ClueNtupleReader

from ml.dynamic_reduction_network import DynamicReductionNetwork
%aimport ml.regression.drn.modules
%aimport ml.regression.drn.plot
%aimport ml.regression.drn.dataset_making
from ml.regression.drn.modules import *
from ml.regression.drn.dataset_making import *
%aimport ml.regression.drn.callbacks
from ml.regression.drn.callbacks import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2023-06-16 17:15:05.702736: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-16 17:15:06.125243: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/grid_mnt/vol_home/llr/cms/cuisset/mambaforge/envs/clustering-analysis-gpu2/lib/python3.10/site-packages/zfit/__init__.py:63: UserWarning: TensorFlow warnings are by default suppressed by zfit. In order to show them, set the environment variable ZFIT_DISABLE_TF_WARNINGS=0. In order to suppress the TensorFlow warnings AND thi

In [4]:
from lightning.pytorch.tuner.tuning import Tuner

In [7]:
reader = ClueNtupleReader("v40", "cmssw", "sim_proton_v46_patchMIP")
datamodule = DRNDataModule(reader, datasetComputationClass=RechitsTensorMaker, transformFct=None, 
                multiprocess_loader=True, keepOnGpu=False
                )
model = DRNModule(drn=DynamicReductionNetwork(input_dim=4,hidden_dim=20,k=10,output_dim=1,norm=torch.tensor([ 1., 1., 1., 1.])),
    loss=RatioCorrectedLoss([-0.2597882 , -0.24326517,  1.01537901]))

In [6]:
trainer = pl.Trainer(accelerator="gpu", devices=[2])

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


## Tuning for maw batch size

In [ ]:
tuner = Tuner()
tuner.scale_batch_size(model=model, datamodule=datamodule, init_val=1024)

## Putting whole dataset on GPU

In [9]:
datamodule.prepare_data()
datamodule.dataset_train_val._data.to("cuda:2")

Data(x=[137226354, 4], beamEnergy=[247493], trueBeamEnergy=[247493], tracksterEnergy=[247493])

In [11]:
datamodule.dataset_train_val[5].x

tensor([[-9.7176e-01,  1.6831e+00,  1.0000e+00,  8.7044e-02],
        [-1.9435e+00,  3.3663e+00,  1.0000e+00,  4.2554e-02],
        [-1.9435e+00,  2.2442e+00,  1.0000e+00,  4.3660e-01],
        ...,
        [-2.9153e+00,  2.8052e+00,  2.7000e+01,  1.0746e-01],
        [ 2.9153e+00,  5.6105e-01,  2.7000e+01,  2.3821e-02],
        [ 1.9435e+00,  1.1221e+00,  2.7000e+01,  2.0941e-02]], device='cuda:2')